In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import math

from matplotlib.pyplot import savefig

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sp

In [2]:
import matplotlib.ticker as ticker
from tqdm.notebook import trange

In [3]:
sensor = pd.read_csv('data/sensor_infomation.csv')
sensor

,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4
0,715933,5,N,7,37,14974.0,10.33,126.963,33.981839,-118.130679,0.425,ML,4,GREENWOOD,2610,NaN,NaN,NaN
1,715938,5,N,7,37,14974.0,12.18,128.813,34.002541,-118.150997,0.545,ML,4,GASPAR,2021,NaN,NaN,NaN
2,715944,5,N,7,37,14974.0,13.35,129.983,34.013676,-118.166091,0.323,ML,4,FERRIS,2537,NaN,NaN,NaN
3,715947,5,S,7,37,14974.0,13.67,130.240,34.015325,-118.171270,0.495,ML,3,S OF 710,2536,NaN,NaN,NaN
4,716010,5,S,7,37,69088.0,R49.5,165.847,34.368149,-118.558850,0.600,ML,4,S OF WABUSKA,2089,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,774672,110,N,7,37,44000.0,21.7,21.630,34.041951,-118.273107,0.370,ML,3,14TH STREET,2608,NaN,NaN,NaN
146,774728,10,W,7,37,58072.0,47.2,45.700,34.081720,-117.728629,0.335,ML,4,E/O SAN ANTONIO,2129,NaN,NaN,NaN
147,775155,405,N,7,37,NaN,31.3,55.072,34.052632,-118.449662,0.310,ML,6,NO OF SANTA MONICA,2061,NaN,NaN,NaN
148,775610,110,S,7,37,44000.0,23.8,23.730,34.063457,-118.247963,0.705,ML,3,SUNSET,2392,NaN,NaN,NaN


In [4]:
training_flow = pd.read_csv('data/training_flow.csv').iloc[:,1:]
training_speed = pd.read_csv('data/training_speed.csv').iloc[:,1:]
test_flow = pd.read_csv('data/test_flow.csv').iloc[:,1:]
test_speed = pd.read_csv('data/test_speed.csv').iloc[:,1:]

In [5]:
training_set_q = training_flow.values
test_set_q = test_flow.values

training_set_v = training_speed.values
test_set_v = test_speed.values

training_set_k = training_set_q/training_set_v
test_set_k = test_set_q/test_set_v

num_sensor = training_set_q.shape[1]
training_set_k.shape

(69695, 150)

In [7]:
s3_v_f = pd.read_csv('parameter/v_f.csv').values
s3_k_c = pd.read_csv('parameter/k_c.csv').values
s3_mm = pd.read_csv('parameter/mm.csv').values

gb_k_jam = pd.read_csv('parameter/GB_k_jam.csv').values
gb_v_c = pd.read_csv('parameter/GB_v_c.csv').values

gs_k_jam = pd.read_csv('parameter/GS_k_jam.csv').values
gs_v_f = pd.read_csv('parameter/GS_v_f.csv').values

In [8]:
def plot_figure(x_train, y_train, x_test, y_test,
                x_s3, y_s3, x_gb, y_gb, x_gs, y_gs,
                label, test_index, 
                xlabel, ylabel, title, majorLocator):
    
    plt.subplot(1, 3, test_index)
    
    plt.scatter(x_train, y_train, s=5, marker='o', label=label[0], facecolors='none', edgecolors=(0.3, 0.6, 0.9))
    
    plt.scatter(x_test, y_test, c='darkorange', s=5, marker='+', label=label[1])
    
    plt.plot(x_s3, y_s3, c='red', linewidth=2, label='S3')
    plt.plot(x_gb, y_gb, c='#006400', linewidth=2, linestyle='--', label='Greenberg')
    plt.plot(x_gs, y_gs, c='black', linewidth=2, linestyle='-.', label='Greenshields')
    
    # Set the label and title of the subgraph
    plt.xlabel(xlabel, fontsize=20)
    plt.ylabel(ylabel, fontsize=20)
    plt.title(title, fontsize=20)
    
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    
    majorLocatorX = ticker.MultipleLocator(majorLocator[0])
    majorLocatorY = ticker.MultipleLocator(majorLocator[1])
    plt.gca().xaxis.set_major_locator(majorLocatorX)
    plt.gca().yaxis.set_major_locator(majorLocatorY)
    
    plt.legend(fontsize=18, loc='best')
    plt.legend(prop={'size': 12})  # 设置图例字体大小为12

    plt.grid(True, color='gray', linestyle='--', linewidth=0.5, which='major')
    
    title_name = 'Sensor ' + "%d" % sensor.loc[i,'ID'] + ' on Freeway ' + "%d" % sensor.loc[i,'Fwy']
    plt.suptitle(title_name, fontsize=20)

In [10]:
for i in trange(num_sensor):
    plt.figure(figsize=(22,5))
    plt.rc('font',family='Times New Roman')
    
    q_train = training_set_q[:,i]
    v_train = training_set_v[:,i]
    k_train = training_set_k[:,i]

    q_test = test_set_q[:,i]
    v_test = test_set_v[:,i]
    k_test = test_set_k[:,i]

    # Speed vs. flow
    y1_s3 = np.linspace(0, s3_v_f[i], 10000, endpoint=True, retstep=False)
    x1_s3 = y1_s3 * s3_k_c[i] * (((s3_v_f[i]**s3_mm[i])/(y1_s3**s3_mm[i]))**0.5-1)**(1/s3_mm[i])
    x1_s3[0,-1] = 0
    
    y1_gb = np.linspace(0, s3_v_f[i]+10, 10000, endpoint=True, retstep=False)
    x1_gb = math.e**(np.log(y1_gb*gb_k_jam[i])-y1_gb/gb_v_c[i])
    
    y1_gs = np.linspace(0, s3_v_f[i]+10, 10000, endpoint=True, retstep=False)
    x1_gs = y1_gs*gs_k_jam[i] - (gs_k_jam[i]*y1_gs**2)/gs_v_f[i]
    
    mask0 = x1_gs >= 0
    x1_gs = x1_gs[mask0]
    y1_gs = y1_gs[mask0]
    
    plot_figure(q_train, v_train, q_test, v_test, x1_s3, y1_s3, x1_gb, y1_gb, x1_gs, y1_gs,['Training set','Test set'],
                1,'Flow (veh/hr/ln)','Speed (mi/hr)','Speed vs. flow',[300,10])
    
    # Speed vs. density
    x2_s3 = np.linspace(0, k_train.max()+10, 10000, endpoint=True, retstep=False)
    y2_s3 = s3_v_f[i]/(1+(x2_s3/s3_k_c[i])**s3_mm[i])**(2/s3_mm[i])
    
    x2_gb = x2_s3
    y2_gb = gb_v_c[i]*np.log(gb_k_jam[i]/x2_gb)
    
    mask1 = (y2_gb >= 0) & (y2_gb <= training_set_v.max())
    x2_gb = x2_gb[mask1]
    y2_gb = y2_gb[mask1]
    
    x2_gs = x2_s3
    y2_gs = gs_v_f[i]*(1-x2_gs/gs_k_jam[i])
    
    mask2 = (y2_gs >= 0) & (y2_gs <= training_set_v.max())
    x2_gs = x2_gs[mask2]
    y2_gs = y2_gs[mask2]
    
    plot_figure(k_train, v_train, k_test, v_test, x2_s3, y2_s3, x2_gb, y2_gb, x2_gs, y2_gs,['Training set','Test set'],
                2,'Density (veh/ln/mi)','Speed (mi/hr)','Speed vs. density',[20,10])
    
    # Flow vs. density
    x3_s3 = np.linspace(0, k_train.max()+10, 10000, endpoint=True, retstep=False)
    y3_s3 = x3_s3*s3_v_f[i]/(1+(x3_s3/s3_k_c[i])**s3_mm[i])**(2/s3_mm[i])
    
    x3_gb = x3_s3
    y3_gb = x3_gb*gb_v_c[i]*np.log(gb_k_jam[i]/x3_gb)
    
    mask3 = (y3_gb >= 0) & (y3_gb <= training_set_q.max())
    x3_gb = x3_gb[mask3]
    y3_gb = y3_gb[mask3]
    
    x3_gs = x3_s3
    y3_gs = x3_gs*gs_v_f[i]*(1-x3_gs/gs_k_jam[i])
    
    mask4 = (y3_gs >= 0) & (y3_gs <= training_set_q.max())
    x3_gs = x3_gs[mask4]
    y3_gs = y3_gs[mask4]
    
    plot_figure(k_train, q_train, k_test, q_test, x3_s3, y3_s3, x3_gb, y3_gb, x3_gs, y3_gs,['Training set','Test set'],
                3,'Density (veh/ln/mi)','Flow (veh/hr/ln)','Flow vs. density',[20,300])
    
    plt.tight_layout()
    
    filename = "%d.png" % sensor.loc[i,'ID']
    file_path = 'figure/FD/' + filename
    plt.savefig(file_path,dpi=300, pad_inches=0.05)
    
    plt.clf()
    plt.close()

  0%|          | 0/150 [00:00<?, ?it/s]

<ipython-input-10-179892b063f8>:15: RuntimeWarning: divide by zero encountered in true_divide
  x1_s3 = y1_s3 * s3_k_c[i] * (((s3_v_f[i]**s3_mm[i])/(y1_s3**s3_mm[i]))**0.5-1)**(1/s3_mm[i])
<ipython-input-10-179892b063f8>:15: RuntimeWarning: invalid value encountered in multiply
  x1_s3 = y1_s3 * s3_k_c[i] * (((s3_v_f[i]**s3_mm[i])/(y1_s3**s3_mm[i]))**0.5-1)**(1/s3_mm[i])
<ipython-input-10-179892b063f8>:19: RuntimeWarning: divide by zero encountered in log
  x1_gb = math.e**(np.log(y1_gb*gb_k_jam[i])-y1_gb/gb_v_c[i])
<ipython-input-10-179892b063f8>:36: RuntimeWarning: divide by zero encountered in true_divide
  y2_gb = gb_v_c[i]*np.log(gb_k_jam[i]/x2_gb)
<ipython-input-10-179892b063f8>:57: RuntimeWarning: divide by zero encountered in true_divide
  y3_gb = x3_gb*gb_v_c[i]*np.log(gb_k_jam[i]/x3_gb)
<ipython-input-10-179892b063f8>:57: RuntimeWarning: invalid value encountered in multiply
  y3_gb = x3_gb*gb_v_c[i]*np.log(gb_k_jam[i]/x3_gb)
<ipython-input-10-179892b063f8>:15: RuntimeWarning

<ipython-input-10-179892b063f8>:15: RuntimeWarning: divide by zero encountered in true_divide
  x1_s3 = y1_s3 * s3_k_c[i] * (((s3_v_f[i]**s3_mm[i])/(y1_s3**s3_mm[i]))**0.5-1)**(1/s3_mm[i])
<ipython-input-10-179892b063f8>:15: RuntimeWarning: invalid value encountered in multiply
  x1_s3 = y1_s3 * s3_k_c[i] * (((s3_v_f[i]**s3_mm[i])/(y1_s3**s3_mm[i]))**0.5-1)**(1/s3_mm[i])
<ipython-input-10-179892b063f8>:19: RuntimeWarning: divide by zero encountered in log
  x1_gb = math.e**(np.log(y1_gb*gb_k_jam[i])-y1_gb/gb_v_c[i])
<ipython-input-10-179892b063f8>:36: RuntimeWarning: divide by zero encountered in true_divide
  y2_gb = gb_v_c[i]*np.log(gb_k_jam[i]/x2_gb)
<ipython-input-10-179892b063f8>:57: RuntimeWarning: divide by zero encountered in true_divide
  y3_gb = x3_gb*gb_v_c[i]*np.log(gb_k_jam[i]/x3_gb)
<ipython-input-10-179892b063f8>:57: RuntimeWarning: invalid value encountered in multiply
  y3_gb = x3_gb*gb_v_c[i]*np.log(gb_k_jam[i]/x3_gb)
<ipython-input-10-179892b063f8>:15: RuntimeWarning

<ipython-input-10-179892b063f8>:57: RuntimeWarning: divide by zero encountered in true_divide
  y3_gb = x3_gb*gb_v_c[i]*np.log(gb_k_jam[i]/x3_gb)
<ipython-input-10-179892b063f8>:57: RuntimeWarning: invalid value encountered in multiply
  y3_gb = x3_gb*gb_v_c[i]*np.log(gb_k_jam[i]/x3_gb)
<ipython-input-10-179892b063f8>:15: RuntimeWarning: divide by zero encountered in true_divide
  x1_s3 = y1_s3 * s3_k_c[i] * (((s3_v_f[i]**s3_mm[i])/(y1_s3**s3_mm[i]))**0.5-1)**(1/s3_mm[i])
<ipython-input-10-179892b063f8>:15: RuntimeWarning: invalid value encountered in multiply
  x1_s3 = y1_s3 * s3_k_c[i] * (((s3_v_f[i]**s3_mm[i])/(y1_s3**s3_mm[i]))**0.5-1)**(1/s3_mm[i])
<ipython-input-10-179892b063f8>:19: RuntimeWarning: divide by zero encountered in log
  x1_gb = math.e**(np.log(y1_gb*gb_k_jam[i])-y1_gb/gb_v_c[i])
<ipython-input-10-179892b063f8>:36: RuntimeWarning: divide by zero encountered in true_divide
  y2_gb = gb_v_c[i]*np.log(gb_k_jam[i]/x2_gb)
<ipython-input-10-179892b063f8>:57: RuntimeWarning

<ipython-input-10-179892b063f8>:15: RuntimeWarning: divide by zero encountered in true_divide
  x1_s3 = y1_s3 * s3_k_c[i] * (((s3_v_f[i]**s3_mm[i])/(y1_s3**s3_mm[i]))**0.5-1)**(1/s3_mm[i])
<ipython-input-10-179892b063f8>:15: RuntimeWarning: invalid value encountered in multiply
  x1_s3 = y1_s3 * s3_k_c[i] * (((s3_v_f[i]**s3_mm[i])/(y1_s3**s3_mm[i]))**0.5-1)**(1/s3_mm[i])
<ipython-input-10-179892b063f8>:19: RuntimeWarning: divide by zero encountered in log
  x1_gb = math.e**(np.log(y1_gb*gb_k_jam[i])-y1_gb/gb_v_c[i])
<ipython-input-10-179892b063f8>:36: RuntimeWarning: divide by zero encountered in true_divide
  y2_gb = gb_v_c[i]*np.log(gb_k_jam[i]/x2_gb)
<ipython-input-10-179892b063f8>:57: RuntimeWarning: divide by zero encountered in true_divide
  y3_gb = x3_gb*gb_v_c[i]*np.log(gb_k_jam[i]/x3_gb)
<ipython-input-10-179892b063f8>:57: RuntimeWarning: invalid value encountered in multiply
  y3_gb = x3_gb*gb_v_c[i]*np.log(gb_k_jam[i]/x3_gb)
<ipython-input-10-179892b063f8>:15: RuntimeWarning

...

<ipython-input-10-179892b063f8>:15: RuntimeWarning: divide by zero encountered in true_divide
  x1_s3 = y1_s3 * s3_k_c[i] * (((s3_v_f[i]**s3_mm[i])/(y1_s3**s3_mm[i]))**0.5-1)**(1/s3_mm[i])
<ipython-input-10-179892b063f8>:15: RuntimeWarning: invalid value encountered in multiply
  x1_s3 = y1_s3 * s3_k_c[i] * (((s3_v_f[i]**s3_mm[i])/(y1_s3**s3_mm[i]))**0.5-1)**(1/s3_mm[i])
<ipython-input-10-179892b063f8>:19: RuntimeWarning: divide by zero encountered in log
  x1_gb = math.e**(np.log(y1_gb*gb_k_jam[i])-y1_gb/gb_v_c[i])
<ipython-input-10-179892b063f8>:36: RuntimeWarning: divide by zero encountered in true_divide
  y2_gb = gb_v_c[i]*np.log(gb_k_jam[i]/x2_gb)
<ipython-input-10-179892b063f8>:57: RuntimeWarning: divide by zero encountered in true_divide
  y3_gb = x3_gb*gb_v_c[i]*np.log(gb_k_jam[i]/x3_gb)
<ipython-input-10-179892b063f8>:57: RuntimeWarning: invalid value encountered in multiply
  y3_gb = x3_gb*gb_v_c[i]*np.log(gb_k_jam[i]/x3_gb)
<ipython-input-10-179892b063f8>:15: RuntimeWarning

<ipython-input-10-179892b063f8>:15: RuntimeWarning: divide by zero encountered in true_divide
  x1_s3 = y1_s3 * s3_k_c[i] * (((s3_v_f[i]**s3_mm[i])/(y1_s3**s3_mm[i]))**0.5-1)**(1/s3_mm[i])
<ipython-input-10-179892b063f8>:15: RuntimeWarning: invalid value encountered in multiply
  x1_s3 = y1_s3 * s3_k_c[i] * (((s3_v_f[i]**s3_mm[i])/(y1_s3**s3_mm[i]))**0.5-1)**(1/s3_mm[i])
<ipython-input-10-179892b063f8>:19: RuntimeWarning: divide by zero encountered in log
  x1_gb = math.e**(np.log(y1_gb*gb_k_jam[i])-y1_gb/gb_v_c[i])
<ipython-input-10-179892b063f8>:36: RuntimeWarning: divide by zero encountered in true_divide
  y2_gb = gb_v_c[i]*np.log(gb_k_jam[i]/x2_gb)
<ipython-input-10-179892b063f8>:57: RuntimeWarning: divide by zero encountered in true_divide
  y3_gb = x3_gb*gb_v_c[i]*np.log(gb_k_jam[i]/x3_gb)
<ipython-input-10-179892b063f8>:57: RuntimeWarning: invalid value encountered in multiply
  y3_gb = x3_gb*gb_v_c[i]*np.log(gb_k_jam[i]/x3_gb)
<ipython-input-10-179892b063f8>:15: RuntimeWarning

<ipython-input-10-179892b063f8>:15: RuntimeWarning: divide by zero encountered in true_divide
  x1_s3 = y1_s3 * s3_k_c[i] * (((s3_v_f[i]**s3_mm[i])/(y1_s3**s3_mm[i]))**0.5-1)**(1/s3_mm[i])
<ipython-input-10-179892b063f8>:15: RuntimeWarning: invalid value encountered in multiply
  x1_s3 = y1_s3 * s3_k_c[i] * (((s3_v_f[i]**s3_mm[i])/(y1_s3**s3_mm[i]))**0.5-1)**(1/s3_mm[i])
<ipython-input-10-179892b063f8>:19: RuntimeWarning: divide by zero encountered in log
  x1_gb = math.e**(np.log(y1_gb*gb_k_jam[i])-y1_gb/gb_v_c[i])
<ipython-input-10-179892b063f8>:36: RuntimeWarning: divide by zero encountered in true_divide
  y2_gb = gb_v_c[i]*np.log(gb_k_jam[i]/x2_gb)
<ipython-input-10-179892b063f8>:57: RuntimeWarning: divide by zero encountered in true_divide
  y3_gb = x3_gb*gb_v_c[i]*np.log(gb_k_jam[i]/x3_gb)
<ipython-input-10-179892b063f8>:57: RuntimeWarning: invalid value encountered in multiply
  y3_gb = x3_gb*gb_v_c[i]*np.log(gb_k_jam[i]/x3_gb)
<ipython-input-10-179892b063f8>:15: RuntimeWarning

<ipython-input-10-179892b063f8>:15: RuntimeWarning: divide by zero encountered in true_divide
  x1_s3 = y1_s3 * s3_k_c[i] * (((s3_v_f[i]**s3_mm[i])/(y1_s3**s3_mm[i]))**0.5-1)**(1/s3_mm[i])
<ipython-input-10-179892b063f8>:15: RuntimeWarning: invalid value encountered in multiply
  x1_s3 = y1_s3 * s3_k_c[i] * (((s3_v_f[i]**s3_mm[i])/(y1_s3**s3_mm[i]))**0.5-1)**(1/s3_mm[i])
<ipython-input-10-179892b063f8>:19: RuntimeWarning: divide by zero encountered in log
  x1_gb = math.e**(np.log(y1_gb*gb_k_jam[i])-y1_gb/gb_v_c[i])
<ipython-input-10-179892b063f8>:36: RuntimeWarning: divide by zero encountered in true_divide
  y2_gb = gb_v_c[i]*np.log(gb_k_jam[i]/x2_gb)
<ipython-input-10-179892b063f8>:57: RuntimeWarning: divide by zero encountered in true_divide
  y3_gb = x3_gb*gb_v_c[i]*np.log(gb_k_jam[i]/x3_gb)
<ipython-input-10-179892b063f8>:57: RuntimeWarning: invalid value encountered in multiply
  y3_gb = x3_gb*gb_v_c[i]*np.log(gb_k_jam[i]/x3_gb)
<ipython-input-10-179892b063f8>:15: RuntimeWarning

<ipython-input-10-179892b063f8>:57: RuntimeWarning: invalid value encountered in multiply
  y3_gb = x3_gb*gb_v_c[i]*np.log(gb_k_jam[i]/x3_gb)
<ipython-input-10-179892b063f8>:15: RuntimeWarning: divide by zero encountered in true_divide
  x1_s3 = y1_s3 * s3_k_c[i] * (((s3_v_f[i]**s3_mm[i])/(y1_s3**s3_mm[i]))**0.5-1)**(1/s3_mm[i])
<ipython-input-10-179892b063f8>:15: RuntimeWarning: invalid value encountered in multiply
  x1_s3 = y1_s3 * s3_k_c[i] * (((s3_v_f[i]**s3_mm[i])/(y1_s3**s3_mm[i]))**0.5-1)**(1/s3_mm[i])
<ipython-input-10-179892b063f8>:19: RuntimeWarning: divide by zero encountered in log
  x1_gb = math.e**(np.log(y1_gb*gb_k_jam[i])-y1_gb/gb_v_c[i])
<ipython-input-10-179892b063f8>:36: RuntimeWarning: divide by zero encountered in true_divide
  y2_gb = gb_v_c[i]*np.log(gb_k_jam[i]/x2_gb)
<ipython-input-10-179892b063f8>:57: RuntimeWarning: divide by zero encountered in true_divide
  y3_gb = x3_gb*gb_v_c[i]*np.log(gb_k_jam[i]/x3_gb)
<ipython-input-10-179892b063f8>:57: RuntimeWarning